# Legal Document Ingestion Pipeline

This notebook demonstrates the complete document ingestion pipeline for a legal assistant system. It processes legal cases and legislation documents, extracting text and metadata to create searchable vector embeddings using ChromaDB and LlamaIndex.

## Overview

The pipeline performs the following key operations:

1. **Document Processing**: Extracts text from PDF documents using PyMuPDF
2. **Metadata Integration**: Combines document content with structured metadata from JSON files
3. **Text Chunking**: Uses semantic splitting to create meaningful document chunks
4. **Vector Embeddings**: Generates embeddings using BAAI/bge-m3 model
5. **Vector Storage**: Stores embeddings in ChromaDB for efficient retrieval

## Document Categories

- **Legal Cases**: Court decisions and case law from 1,580 PDF files (originally from 16 folders with 100 files each, minus 20 test cases)
- **Legislation**: Various types of legal documents including acts, amendments, ordinances, and federal constitution

---

## Table of Contents

1. [Setup Python Environment](#1-setup-python-environment)
2. [Import Required Libraries](#2-import-required-libraries)
3. [Configuration Parameters](#3-configuration-parameters)
4. [Initialize Embedding Model](#4-initialize-embedding-model)
5. [Set Up Vector Database (ChromaDB)](#5-set-up-vector-database-chromadb)
   - 5.1 [Create Document Collections](#51-create-document-collections)
   - 5.2 [Initialize Vector Stores](#52-initialize-vector-stores)
6. [Configure Document Processing Pipeline](#6-configure-document-processing-pipeline)
   - 6.1 [Semantic Text Splitter](#61-semantic-text-splitter)
   - 6.2 [Ingestion Pipelines](#62-ingestion-pipelines)
7. [Process Legal Cases Documents](#7-process-legal-cases-documents)
   - 7.1 [Document Structure](#71-document-structure)
   - 7.2 [Processing Workflow](#72-processing-workflow)
   - 7.3 [Error Handling](#73-error-handling)
8. [Process Legislation Documents](#8-process-legislation-documents)
   - 8.1 [Document Categories](#81-document-categories)
   - 8.2 [Processing Workflow](#82-processing-workflow)
   - 8.3 [Key Features](#83-key-features)
9. [Conclusion](#conclusion)
   - [Next Steps](#next-steps)
   - [Database Statistics](#database-statistics)

---

## 1. Setup Python Environment

This section configures the Python environment to properly import local modules from the project structure. Since this notebook is located in a subdirectory (`app/api/src/preprocessing/`), we need to add the project root to the Python path to enable imports from the `app` module.

### Key Setup Steps:
- **Project Root Detection**: Automatically finds the project root directory
- **Python Path Configuration**: Adds the root to `sys.path` for proper module resolution
- **Import Enablement**: Allows importing from `app.api.src.*` modules

This step is **essential** and must be run before any other imports.

In [14]:
# Setup Python path to include project root
import sys
import os
from pathlib import Path

# Add project root to Python path
project_root = Path(__file__).parent.parent.parent.parent.absolute() if '__file__' in globals() else Path.cwd().parent.parent.parent.parent
sys.path.insert(0, str(project_root))

print(f"📁 Project root: {project_root}")
print(f"🐍 Python path updated to include project root")
print()

📁 Project root: c:\Work and School\project\llm-legal-assistant
🐍 Python path updated to include project root



## 2. Import Required Libraries

This section imports all necessary libraries for the document ingestion pipeline:

- **LlamaIndex Components**: 
  - `HuggingFaceEmbedding`: For generating vector embeddings from text
  - `FlagEmbeddingReranker`: For reranking search results (imported but not used in this notebook)
  - `ChromaVectorStore`: Interface to ChromaDB vector database
  - `SemanticSplitterNodeParser`: For intelligent text chunking
  - `IngestionPipeline`: Orchestrates the document processing workflow
  - `Document`: Core document object for storing text and metadata

- **Local Application Modules**:
  - `db_config`: Configuration settings for database connections

- **Third-party Libraries**:
  - `chromadb`: Vector database for storing embeddings
  - `pymupdf4llm`: PDF text extraction using PyMuPDF
  - `pathlib.Path`: For file system operations

- **Standard Library**:
  - `os`: File system operations
  - `json`: JSON metadata processing

In [25]:
# LlamaIndex imports
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import Document

# Local application imports
from app.api.src.storage.db_config import db_config

# Third-party imports
import chromadb
import pymupdf4llm
from pathlib import Path
from tqdm import tqdm
import time

# Standard library imports
import os
import json

## 3. Configuration Parameters

This section defines the key configuration parameters for the ingestion pipeline:

- **`vdb_dir`**: Directory path for storing the ChromaDB vector database
- **`embed_model_name`**: HuggingFace model name for text embeddings (BAAI/bge-m3)
- **`rerank_model_name`**: Model for reranking search results (BAAI/bge-reranker-large)
- **`device`**: Computing device for model inference ('cuda' for GPU acceleration)

Directory paths are defined directly in the processing sections for better clarity.

In [16]:
# Configuration parameters for the ingestion pipeline
vdb_dir = "./vector_db"  # Local directory for ChromaDB storage
embed_model_name: str = "BAAI/bge-m3"  # Multilingual embedding model
rerank_model_name: str = "BAAI/bge-reranker-large"  # Model for reranking (future use)
device = "cuda"  # Use GPU for faster processing

print("📋 Configuration loaded:")
print(f"   📂 Vector DB directory: {vdb_dir}")
print(f"   🤖 Embedding model: {embed_model_name}")
print(f"   🔄 Reranker model: {rerank_model_name}")
print(f"   💻 Device: {device}")

📋 Configuration loaded:
   📂 Vector DB directory: ./vector_db
   🤖 Embedding model: BAAI/bge-m3
   🔄 Reranker model: BAAI/bge-reranker-large
   💻 Device: cuda


In [17]:
def load_document_json(path: str) -> Document:
    """
    Load a Document object back from JSON metadata file
    
    This function reads a JSON metadata file and creates a LlamaIndex Document
    object with the text content and associated metadata.
    
    Args:
        path (str): Absolute path to the JSON metadata file
    
    Returns:
        Document: LlamaIndex Document object with text and metadata
    
    Example:
        doc = load_document_json("./metadata/case_document.json")
        print(f"Document text: {doc.text[:100]}...")
        print(f"Case number: {doc.metadata.get('case_number', 'N/A')}")
    """
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return Document(text=data["text"], metadata=data["metadata"])

## 4. Initialize Embedding Model

Initialize the HuggingFace embedding model that will be used to convert text into vector representations. The BAAI/bge-m3 model is a multilingual embedding model that supports:

- **Multiple Languages**: Optimized for various languages including English
- **High Quality**: State-of-the-art performance for retrieval tasks
- **GPU Acceleration**: Runs on CUDA-enabled devices for faster processing

In [18]:
embed_model = HuggingFaceEmbedding(
    model_name=embed_model_name,
    device=device,
    embed_batch_size=8 # Lower batch size to lowers peak VRAM use and avoids fragmentation.
)

print("Embedding model initialized successfully")
test = embed_model.get_text_embedding("warmup")
print(test)
print("Embedding model warmed up")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-m3


INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']


Embedding model initialized successfully
[-0.005207625217735767, -0.0006870775832794607, -0.02223331294953823, 0.020184507593512535, -0.029927697032690048, -0.06711073964834213, 0.031456220895051956, 0.0159471333026886, 0.023328598588705063, -0.02996118739247322, 0.01998835802078247, 0.0009859318379312754, -0.018612220883369446, -0.017339635640382767, 0.017828110605478287, -0.02690240368247032, -0.0059281992726027966, 0.008089065551757812, 0.01929176226258278, -0.013787437230348587, 0.03635026142001152, -0.026004280894994736, 0.007953831925988197, -0.022007634863257408, 0.0057941642589867115, 0.019207723438739777, 0.0012700532097369432, -0.015034619718790054, 0.0022509668488055468, -0.010847724974155426, 0.011510312557220459, -0.010049857199192047, -0.06183948367834091, -0.08297886699438095, -0.037011608481407166, -0.05616115406155586, -0.021723266690969467, -0.0031647798605263233, -0.06387408822774887, -0.012561041861772537, -0.0010800358140841126, -0.008124255575239658, 0.01354601047

## 5. Set Up Vector Database (ChromaDB)

Configure ChromaDB as the vector database for storing document embeddings. ChromaDB provides:

- **Persistent Storage**: Data is saved locally and persists between sessions
- **Efficient Retrieval**: Fast similarity search using vector embeddings
- **Metadata Support**: Stores both embeddings and associated metadata
- **Scalability**: Handles large document collections efficiently

In [19]:
# Get ChromaDB configuration from database config
print("🔧 Initializing ChromaDB client...")
client_settings = db_config.chroma.client_settings

if "persist_directory" in client_settings:
    # Local persistent storage configuration
    persist_dir = vdb_dir
    print(f"   📁 Creating directory: {persist_dir}")
    Path(persist_dir).mkdir(parents=True, exist_ok=True)
    
    # Initialize persistent ChromaDB client
    chroma_client = chromadb.PersistentClient(path=persist_dir)
    print(f"   ✅ Using local ChromaDB at: {persist_dir}")
    print(f"   💾 Data will persist between sessions")
else:
    # Fallback to in-memory client if no persistent directory configured
    print("   ⚠️  No persistent directory found, using in-memory client")
    chroma_client = chromadb.Client()

print()

🔧 Initializing ChromaDB client...
   📁 Creating directory: ./vector_db
   ✅ Using local ChromaDB at: ./vector_db
   💾 Data will persist between sessions



### 5.1 Create Document Collections

Create separate collections for different types of legal documents:

- **`legal_cases`**: Stores court decisions, case law, and judicial opinions
- **`legislation`**: Stores acts, amendments, ordinances, and constitutional documents

This separation allows for targeted retrieval and better organization of different document types.

In [47]:
# Create separate collections for different document types
print("🗂️  Creating document collections...")

# Collection for legal cases (court decisions, case law)
legal_cases_collection = chroma_client.get_or_create_collection(
    name="legal_cases",
    metadata={"description": "Legal case documents and metadata"}
)
print(f"   ⚖️  Legal cases collection: '{legal_cases_collection.name}'")
print(f"       📄 Document count: {legal_cases_collection.count()}")

# Collection for legislation (acts, amendments, ordinances, constitution)
legislation_collection = chroma_client.get_or_create_collection(
    name="legislation", 
    metadata={"description": "Legislation documents and metadata"}
)
print(f"   📜 Legislation collection: '{legislation_collection.name}'")
print(f"       📄 Document count: {legislation_collection.count()}")

print(f"   ✅ Collections ready for document ingestion")
print()

🗂️  Creating document collections...
   ⚖️  Legal cases collection: 'legal_cases'
       📄 Document count: 21886
   📜 Legislation collection: 'legislation'
       📄 Document count: 0
   ✅ Collections ready for document ingestion



### 5.2 Initialize Vector Stores

Create ChromaVectorStore instances that bridge the ChromaDB collections with LlamaIndex's vector store interface. These stores handle:

- **Embedding Integration**: Automatically generates embeddings during document insertion
- **Query Processing**: Converts text queries to vector searches
- **Result Formatting**: Returns search results in LlamaIndex-compatible format

In [48]:
# Create vector stores that bridge ChromaDB collections with LlamaIndex
print("🔗 Creating vector store interfaces...")

# Vector store for legal cases collection
legal_cases_vector_store = ChromaVectorStore(
    chroma_collection=legal_cases_collection,
    embed_model=embed_model  # Link to our embedding model
)
print(f"   ⚖️  Legal cases vector store initialized")
print(f"       🔗 Linked to collection: {legal_cases_collection.name}")

# Vector store for legislation collection
legislation_vector_store = ChromaVectorStore(
    chroma_collection=legislation_collection,
    embed_model=embed_model  # Link to our embedding model
)
print(f"   📜 Legislation vector store initialized")
print(f"       🔗 Linked to collection: {legislation_collection.name}")

print(f"   ✅ Vector stores ready for pipeline integration")
print()

🔗 Creating vector store interfaces...
   ⚖️  Legal cases vector store initialized
       🔗 Linked to collection: legal_cases
   📜 Legislation vector store initialized
       🔗 Linked to collection: legislation
   ✅ Vector stores ready for pipeline integration



## 6. Configure Document Processing Pipeline

Set up the document processing pipeline with semantic text splitting and embedding generation.

### 6.1 Semantic Text Splitter

The `SemanticSplitterNodeParser` intelligently chunks documents based on semantic boundaries:

- **`buffer_size=1`**: Minimum number of sentences per chunk
- **`breakpoint_percentile_threshold=95`**: Sensitivity for detecting semantic breaks (95th percentile)
- **`embed_model`**: Uses the same embedding model for consistency

### 6.2 Ingestion Pipelines

Create separate pipelines for each document type, each containing:
1. **Semantic Splitter**: Breaks documents into meaningful chunks
2. **Embedding Model**: Converts text chunks to vector representations
3. **Vector Store**: Stores embeddings in the appropriate ChromaDB collection

In [49]:
# Configure semantic text splitter for intelligent document chunking
print("✂️  Configuring semantic text splitter...")
splitter = TokenTextSplitter(
    chunk_size=1024, # Approx 1024 tokens per chunk
    chunk_overlap=50, # 50 tokens overlap between chunks
    separator="\n\n"   # paragraph-level splitting
)

print(f"   📏 Chunk size: {splitter.chunk_size} sentence(s)")
print(f"   🎯 Chunk overlap: {splitter.chunk_overlap}")
print(f"   ✅ Token text splitter configured")

# Create ingestion pipelines for document processing
print("\n🔄 Setting up ingestion pipelines...")

# Pipeline for legal cases
legal_cases_pipeline = IngestionPipeline(
    transformations=[splitter, embed_model],  # Split text + generate embeddings
    vector_store=legal_cases_vector_store,    # Store in legal cases collection
)
print(f"   ⚖️  Legal cases pipeline: {len(legal_cases_pipeline.transformations)} transformations")

# Pipeline for legislation
legislation_pipeline = IngestionPipeline(
    transformations=[splitter, embed_model],  # Split text + generate embeddings
    vector_store=legislation_vector_store,    # Store in legislation collection
)
print(f"   📜 Legislation pipeline: {len(legislation_pipeline.transformations)} transformations")

print(f"   ✅ Pipelines ready for document processing")
print()

✂️  Configuring semantic text splitter...
   📏 Chunk size: 1024 sentence(s)
   🎯 Chunk overlap: 50
   ✅ Token text splitter configured

🔄 Setting up ingestion pipelines...
   ⚖️  Legal cases pipeline: 2 transformations
   📜 Legislation pipeline: 2 transformations
   ✅ Pipelines ready for document processing



## 7. Process Legal Cases Documents

This section processes legal case documents from a single directory containing PDF files and corresponding JSON metadata.

### 7.1 Document Structure

- **PDF Directory**: `../../../../data/raw/legal_cases/rag_legal_case_files`
- **Metadata Directory**: `../../../../data/raw/legal_cases/rag_legal_case_files/metadata`

### 7.2 Processing Workflow

1. **Directory Validation**: Checks if both PDF and metadata directories exist
2. **File Discovery**: Lists all PDF and JSON files in the directory
3. **File Matching**: Matches PDF files with their corresponding JSON metadata using `extract_base_filename`
4. **Text Extraction**: Extracts text from PDFs using PyMuPDF (pymupdf4llm)
5. **Metadata Integration**: Combines extracted text with JSON metadata
6. **Pipeline Ingestion**: Processes documents through the semantic splitter and embedding pipeline

### 7.3 Error Handling

- Skips directories that don't exist
- Logs warnings for missing files
- Continues processing even if individual files fail
- Provides detailed progress feedback with emojis for better visibility

In [11]:
def extract_base_filename(filename):
    """
    Extract base filename for matching PDF and JSON files
    
    This function normalizes filenames to enable proper matching between
    PDF documents and their corresponding JSON metadata files.
    
    Args:
        filename (str): Full filename with extension
    
    Returns:
        str: Base filename without extension and metadata suffix
    
    Examples:
        "test.pdf" 
        -> "test"
        
        "test_metadata.json"
        -> "test"
        
        "legal_document_v2_metadata.json"
        -> "legal_document_v2"
    """
    # Step 1: Remove file extension (.pdf, .json, etc.)
    base_name = os.path.splitext(filename)[0]
    
    # Step 2: Remove "_metadata" suffix if present (for JSON metadata files)
    if base_name.endswith("_metadata"):
        base_name = base_name[:-9]  # Remove "_metadata" (9 characters)
    
    return base_name

# Test the function with example inputs
print("🔧 Testing filename extraction function:")
test_files = ["document.pdf", "document_metadata.json", "case_law_v1.pdf", "case_law_v1_metadata.json"]
for test_file in test_files:
    result = extract_base_filename(test_file)
    print(f"   '{test_file}' -> '{result}'")
print("   ✅ Function working correctly")
print()

🔧 Testing filename extraction function:
   'document.pdf' -> 'document'
   'document_metadata.json' -> 'document'
   'case_law_v1.pdf' -> 'case_law_v1'
   'case_law_v1_metadata.json' -> 'case_law_v1'
   ✅ Function working correctly



In [12]:
# Initialize PyMuPDF reader for PDF text extraction
print("📚 Initializing document reader...")
llama_reader = pymupdf4llm.LlamaMarkdownReader()
print("   ✅ PyMuPDF reader ready for PDF processing")

# Define directory paths for legal cases
print("\n📁 Setting up directory paths for legal cases...")
rag_legal_cases_dir = "../../../../data/processed/legal_cases/processed_rag_legal_case_files"

print(f"   📂 JSON files directory: {rag_legal_cases_dir}")

# Storage for all processed documents
all_documents = []
print(f"\n🚀 Starting legal cases processing...")

def sanitize_metadata(metadata: dict, max_len: int = 500) -> dict:
    clean_meta = {}
    for k, v in metadata.items():
        if isinstance(v, (str, int, float)) or v is None:
            val = v
        else:
            val = json.dumps(v, ensure_ascii=False)
        
        # Truncate overly long metadata values
        if isinstance(val, str) and len(val) > max_len:
            val = val[:max_len] + "…"

        clean_meta[k] = val
    return clean_meta

# Check if PDF directory exists
if not os.path.exists(rag_legal_cases_dir):
    print(f"   ❌ JSON directory not found: {rag_legal_cases_dir}")
else:
    # Get list of JSON files
    json_files = [f for f in os.listdir(rag_legal_cases_dir) if f.endswith('.json')]
    
    # Validate that we have JSON files
    if not json_files:
        print(f"   ⚠️ No JSON files found in {os.path.basename(rag_legal_cases_dir)}")
    else:
        print(f"   📄 Found {len(json_files)} JSON files to process")
        
        # Process each JSON file in the directory
        processed_count = 0
        for json_file in json_files:
            json_path = os.path.join(rag_legal_cases_dir, json_file)
            
            try:
                # Load the pre-processed JSON file
                print(f"   🔍 Loading: {json_file}")
                
                # Load and parse JSON data
                with open(json_path, "r", encoding="utf-8") as f:
                    json_data = json.load(f)
                    
                clean_metadata = sanitize_metadata(json_data["metadata"])
                
                # Create Document object from JSON data
                # The JSON already contains both 'text' and 'metadata'
                document = Document(
                    text=json_data["text"],
                    metadata=clean_metadata
                )
                
                # Ensure consistent metadata fields
                document.metadata["document_type"] = "legal_case"
                document.metadata["source_type"] = "processed_json"
                
                # Add document to the collection
                all_documents.append(document)
                processed_count += 1
                print(f"   ✅ Loaded document from {json_file}")
                
            except Exception as e:
                print(f"   ❌ Failed to process {json_file}: {str(e)}")
        
        # Summary for processing
        print(f"   📊 Processing summary: {processed_count}/{len(json_files)} files processed successfully")

📚 Initializing document reader...
Successfully imported LlamaIndex
   ✅ PyMuPDF reader ready for PDF processing

📁 Setting up directory paths for legal cases...
   📂 JSON files directory: ../../../../data/processed/legal_cases/processed_rag_legal_case_files

🚀 Starting legal cases processing...
   📄 Found 1580 JSON files to process
   🔍 Loading: (M1)_22-203-2006,_(M3)_22-45-2008_(Mahkamah_Tinggi).pdf.json
   ✅ Loaded document from (M1)_22-203-2006,_(M3)_22-45-2008_(Mahkamah_Tinggi).pdf.json
   🔍 Loading: (M1)_22-203-2006_(M3)_22-45-2008_(Mahkamah_Tinggi).pdf.json
   ✅ Loaded document from (M1)_22-203-2006_(M3)_22-45-2008_(Mahkamah_Tinggi).pdf.json
   🔍 Loading: 01(f)-13-09_2021(W)_(Mahkamah_Persekutuan).pdf.json
   ✅ Loaded document from 01(f)-13-09_2021(W)_(Mahkamah_Persekutuan).pdf.json
   🔍 Loading: 01(f)-20-08_2019(Q)_(Mahkamah_Persekutuan)_1.pdf.json
   ✅ Loaded document from 01(f)-20-08_2019(Q)_(Mahkamah_Persekutuan)_1.pdf.json
   🔍 Loading: 01(f)-27-08_2018(Q)_(Mahkamah_Persekut

In [13]:
# Final ingestion step - process all documents through the pipeline
if all_documents:
    print(f"\n🔄 Running ingestion pipeline for {len(all_documents)} documents...")
    print(f"   🔧 Pipeline will: split text → generate embeddings → store in ChromaDB")
    
    # Config
    batch_size = 4  # Smaller batches for GPU stability
    checkpoint_file = "./checkpoint/legal_case_ingestion_checkpoint.json"  # progress file
    total_batches = (len(all_documents) + batch_size - 1) // batch_size

    # Load checkpoint if exists
    start_batch = 0
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, "r", encoding="utf-8") as f:
            checkpoint = json.load(f)
            start_batch = checkpoint.get("last_batch", 0)
            print(f"⏩ Resuming from batch {start_batch+1}/{total_batches}")

    with tqdm(
        total=len(all_documents), 
        desc="🚀 Ingesting Documents", 
        unit="docs",
        bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]'
    ) as pbar:
        
        start_time = time.time()
        pbar.update(start_batch * batch_size)

        for i in range(start_batch * batch_size, len(all_documents), batch_size):
            batch = all_documents[i:i+batch_size]
            batch_num = i // batch_size + 1

            # Update description with current batch info
            pbar.set_description(f"🚀 Processing Batch {batch_num}/{total_batches}")

            try:
                # Process this batch through pipeline
                legal_cases_pipeline.run(documents=batch)
            except Exception as e:
                print(f"❌ Error on batch {batch_num}: {e}")
                print("💾 Progress saved, you can restart safely.")
                break

            # Save checkpoint
            with open(checkpoint_file, "w", encoding="utf-8") as f:
                json.dump({"last_batch": batch_num}, f)

            # Update progress
            pbar.update(len(batch))

            # Free VRAM between batches
            import torch, gc
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

            # Calculate and show additional metrics
            elapsed_time = time.time() - start_time
            docs_per_second = pbar.n / elapsed_time if elapsed_time > 0 else 0
            pbar.set_postfix({
                'Batch': f"{batch_num}/{total_batches}",
                'Speed': f"{docs_per_second:.1f} docs/s",
                'Stage': 'Embedding'
            })
    
    print(f"✅ Successfully ingested {pbar.n} legal cases documents")
    print(f"📊 Documents now available in 'legal_cases' collection")

else:
    print(f"\n⚠️ No documents to process - skipping pipeline ingestion")


🔄 Running ingestion pipeline for 1580 documents...
   🔧 Pipeline will: split text → generate embeddings → store in ChromaDB


🚀 Processing Batch 395/395: 100%|██████████| 1580/1580 [41:22<00:00,  1.57s/docs]

✅ Successfully ingested 1580 legal cases documents
📊 Documents now available in 'legal_cases' collection


In [14]:
# Test Query using proper vector store interface
if legal_cases_collection.count() > 0:
    print("🔍 Testing query functionality with correct embedding model...")
    
    # Create a query engine using the vector store (uses your bge-m3 model)
    from llama_index.core import VectorStoreIndex
    
    # Create index from vector store
    legal_cases_index = VectorStoreIndex.from_vector_store(
        vector_store=legal_cases_vector_store,
        embed_model=embed_model  # This ensures we use YOUR embedding model
    )
    
    # Create query engine
    query_engine = legal_cases_index.as_query_engine(
        similarity_top_k=2
    )
    
    # Test query using the same embedding model as ingestion
    print("   🔍 Running test query with BAAI/bge-m3 model...")
    response = query_engine.query("legal case")
    
    print(f"   ✅ Query successful using {embed_model_name}!")
    print(f"   📊 Response: {response.response[:200]}...")
    print(f"   🎯 Found {len(response.source_nodes)} relevant chunks")
    
    # Show source information
    for i, node in enumerate(response.source_nodes, 1):
        doc_type = node.metadata.get('document_type', 'Unknown')
        source = node.metadata.get('source_filename', 'Unknown')
        print(f"   📄 Source {i}: {doc_type} - {source}")
    
    print()
else:
    print("⚠️  No data to test queries on")

🔍 Testing query functionality with correct embedding model...
   🔍 Running test query with BAAI/bge-m3 model...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


   ✅ Query successful using BAAI/bge-m3!
   📊 Response: The document provided contains information related to a legal case involving Telaga Biru Sdn Bhd as the plaintiff and Karya Bestari Sdn Bhd as the defendant. The case was heard at the Mahkamah Tinggi ...
   🎯 Found 2 relevant chunks
   📄 Source 1: legal_case - 22IP-66-12_2015_(Mahkamah_Tinggi).pdf
   📄 Source 2: legal_case - 22IP-66-12_2015_(Mahkamah_Tinggi).pdf



## 8. Process Legislation Documents

This section processes various types of legislation documents organized in a hierarchical directory structure.

### 8.1 Document Categories

The legislation processing covers six main categories:

1. **Amendments** (`act/amendment/`): Legislative amendments to existing acts
2. **Principal Acts - Updated** (`act/principal/updated/`): Current versions of principal legislation
3. **Principal Acts - Revised** (`act/principal/revised/`): Revised versions of principal legislation
4. **Federal Constitution** (`federal_constitution/`): Constitutional documents
5. **Ordinances** (`ordinance/`): Local ordinances and regulations
6. **Subsidiary Legislation** (`subsidiary_legislation/`): Supporting regulations and rules

### 8.2 Processing Workflow

1. **Directory Iteration**: Processes each legislation category separately
2. **Metadata Mapping**: Creates a lookup dictionary for efficient PDF-JSON matching
3. **File Processing**: Extracts text from PDFs and integrates with metadata
4. **Flexible Matching**: Uses improved matching logic for metadata files
5. **Batch Ingestion**: Processes all legislation documents through the dedicated pipeline

### 8.3 Key Features

- **Robust Directory Handling**: Skips missing directories gracefully
- **Efficient Metadata Lookup**: Uses dictionary mapping for O(1) metadata retrieval
- **Comprehensive Logging**: Provides detailed progress information
- **Error Resilience**: Continues processing even if individual documents fail

The processed documents are stored in the `legislation` ChromaDB collection for efficient retrieval and search.

In [26]:
# Initialize PyMuPDF reader for PDF text extraction
print("📚 Initializing document reader...")
llama_reader = pymupdf4llm.LlamaMarkdownReader()

# Base directory for all legislation documents
print("📜 Setting up legislation document processing...")
base_dir_legislation = "../../../../data/raw/legislation"
print(f"   📁 Base directory: {base_dir_legislation}")

# Define subdirectory for acts
act_dir = os.path.join(base_dir_legislation, "act")

# Define all legislation category directories
print("\n📂 Configuring legislation categories...")
amendment_dir = os.path.join(base_dir_legislation, "act", "amendment")
principal_updated_dir = os.path.join(base_dir_legislation, "act", "principal", "updated")
principal_revised_dir = os.path.join(base_dir_legislation, "act", "principal", "revised")
federal_constitution_dir = os.path.join(base_dir_legislation, "federal_constitution")
ordinance_dir = os.path.join(base_dir_legislation, "ordinance")

# Too much to process
# subsidiary_legislation_dir = os.path.join(base_dir_legislation, "subsidiary_legislation")

# Create list of directories to process
dir_list = [
    amendment_dir,
    principal_updated_dir,
    principal_revised_dir,
    federal_constitution_dir,
    ordinance_dir,
    # subsidiary_legislation_dir
]

# Category names for better logging
category_names = [
    "Amendments",
    "Principal Acts (Updated)",
    "Principal Acts (Revised)", 
    "Federal Constitution",
    "Ordinances",
    # "Subsidiary Legislation"
]

print(f"   📋 Configured {len(dir_list)} legislation categories:")
for i, (dir_path, cat_name) in enumerate(zip(dir_list, category_names), 1):
    exists = "✅" if os.path.exists(dir_path) else "❌"
    print(f"   {exists} {i}. {cat_name}: {os.path.basename(dir_path)}")

# Storage for all legislation documents
all_legislation_documents = []
total_categories_processed = 0

print(f"\n🚀 Starting legislation processing...")

# Process each legislation category directory
for category_idx, dir_path in enumerate(dir_list, 1):
    category_name = category_names[category_idx - 1]
    print(f"\n📂 Category {category_idx}/6: {category_name}")
    print(f"   📁 Directory: {dir_path}")
    
    # Check if main directory and metadata subdirectory exist
    metadata_dir = os.path.join(dir_path, "metadata")
    if not os.path.exists(dir_path) or not os.path.exists(metadata_dir):
        print(f"   ⚠️ Skipping - missing PDF or metadata folder")
        continue

    # Get lists of PDF and JSON files
    pdf_files = [f for f in os.listdir(dir_path) if f.endswith(".pdf")]
    json_files = [f for f in os.listdir(metadata_dir) if f.endswith(".json")]

    # Validate file availability
    if not pdf_files:
        print(f"   ⚠️ No PDFs found in {os.path.basename(dir_path)}")
        continue
    if not json_files:
        print(f"   ⚠️ No JSON metadata found in metadata folder")
        continue

    print(f"   📄 Found {len(pdf_files)} PDFs and {len(json_files)} JSON files")

    # Build efficient lookup dictionary for metadata files
    # This removes '_metadata' suffix to match with PDF base names
    json_map = {os.path.splitext(j)[0].replace("_metadata", ""): j for j in json_files}
    print(f"   🗂️ Created metadata lookup for {len(json_map)} files")

    # Process each PDF file in the category
    processed_in_category = 0
    for pdf_idx, pdf_file in enumerate(pdf_files, 1):
        pdf_path = os.path.join(dir_path, pdf_file)
        base_name = os.path.splitext(pdf_file)[0]  # Remove .pdf extension

        try:
            # Extract text content from PDF
            documents = llama_reader.load_data(pdf_path)

            # Try to find matching JSON metadata file
            if base_name in json_map:
                json_file = json_map[base_name]
                json_path = os.path.join(metadata_dir, json_file)
                print(f"   🔍 [{pdf_idx}/{len(pdf_files)}] Matched: {pdf_file} ↔ {json_file}")

                # Load and parse JSON metadata
                with open(json_path, "r", encoding="utf-8") as f:
                    json_metadata = json.load(f)

                # Update each document with metadata
                for doc in documents:
                    doc.metadata.update(json_metadata)
                    doc.metadata["source_filename"] = pdf_file
                    doc.metadata["document_type"] = "legislation"
                    doc.metadata["category"] = category_name
                    doc.metadata["category_index"] = category_idx
            else:
                print(f"   ⚠️ [{pdf_idx}/{len(pdf_files)}] No metadata found for: {pdf_file}")

            # Add documents to the collection
            all_legislation_documents.extend(documents)
            processed_in_category += 1
            print(f"   ✅ Extracted {len(documents)} document chunks from {pdf_file}")
            
        except Exception as e:
            print(f"   ❌ Failed to process {pdf_file}: {str(e)}")

    # Summary for current category
    print(f"   📊 Category summary: {processed_in_category}/{len(pdf_files)} files processed")
    total_categories_processed += 1

📚 Initializing document reader...
Successfully imported LlamaIndex
📜 Setting up legislation document processing...
   📁 Base directory: ../../../../data/raw/legislation

📂 Configuring legislation categories...
   📋 Configured 5 legislation categories:
   ✅ 1. Amendments: amendment
   ✅ 2. Principal Acts (Updated): updated
   ✅ 3. Principal Acts (Revised): revised
   ✅ 4. Federal Constitution: federal_constitution
   ✅ 5. Ordinances: ordinance

🚀 Starting legislation processing...

📂 Category 1/6: Amendments
   📁 Directory: ../../../../data/raw/legislation\act\amendment
   📄 Found 204 PDFs and 204 JSON files
   🗂️ Created metadata lookup for 204 files
   🔍 [1/204] Matched: A1471.pdf ↔ A1471_metadata.json
   ✅ Extracted 13 document chunks from A1471.pdf
   🔍 [2/204] Matched: A1472.pdf ↔ A1472_metadata.json
   ✅ Extracted 4 document chunks from A1472.pdf
   🔍 [3/204] Matched: A1473.pdf ↔ A1473_metadata.json
   ✅ Extracted 5 document chunks from A1473.pdf
   🔍 [4/204] Matched: A1474.pdf ↔ 

### 8.4 Data Cleaning and Sanitization

Before processing legislation documents through the ingestion pipeline, we implement robust data cleaning to handle:

- **Unicode Issues**: Removes surrogate characters that can cause encoding problems
- **Metadata Size Limits**: Truncates overly long metadata values to prevent storage issues
- **Type Safety**: Ensures all metadata values are valid JSON-serializable types
- **Error Handling**: Gracefully handles malformed text and metadata

This cleaning step is essential for reliable vector storage and retrieval.

In [ ]:
import re

def clean_text(text: str) -> str:
    """
    Remove surrogate and invalid unicode characters safely.
    
    Surrogate characters (U+D800-U+DFFF) are invalid in UTF-8 and can cause
    encoding issues when storing documents in vector databases. This function
    removes them to ensure clean text processing.
    
    Args:
        text (str): Input text that may contain surrogate characters
    
    Returns:
        str: Cleaned text with surrogate characters removed
    
    Example:
        clean_text("Hello\ud800World") -> "HelloWorld"
    """
    if not isinstance(text, str):
        return str(text)
    return re.sub(r'[\ud800-\udfff]', '', text)

def sanitize_metadata(metadata: dict, max_len: int = 500) -> dict:
    """
    Clean and sanitize metadata for safe storage in vector database.
    
    This function ensures that all metadata values are compatible with ChromaDB
    storage requirements by:
    - Converting complex objects to JSON strings
    - Cleaning unicode surrogate characters
    - Truncating overly long values to prevent storage bloat
    - Ensuring type safety for database operations
    
    Args:
        metadata (dict): Raw metadata dictionary with potentially problematic values
        max_len (int): Maximum length for string metadata values (default: 500)
    
    Returns:
        dict: Sanitized metadata ready for vector storage
    
    Example:
        raw_meta = {"title": "Case\ud800Name", "complex_obj": {"nested": "data"}}
        clean_meta = sanitize_metadata(raw_meta)
        # Result: {"title": "CaseName", "complex_obj": '{"nested": "data"}'}
    """
    clean_meta = {}
    for k, v in metadata.items():
        # ✅ Ensure valid types - keep primitives as-is
        if isinstance(v, (str, int, float)) or v is None:
            val = v
        else:
            # Convert complex objects (lists, dicts, etc.) to JSON string
            val = json.dumps(v, ensure_ascii=False)

        # ✅ Always clean surrogate characters if it's a string
        if isinstance(val, str):
            val = clean_text(val)
            # Truncate overly long metadata values to prevent storage issues
            if len(val) > max_len:
                val = val[:max_len] + "…"

        clean_meta[k] = val
    return clean_meta

# Note: The cleaning functions above are applied during batch processing
# in the legislation ingestion pipeline to ensure data quality and compatibility

### 8.5 Enhanced Batch Processing with Data Cleaning

The legislation ingestion pipeline now includes real-time data cleaning during batch processing. This approach ensures that:

- **Memory Efficiency**: Cleaning is applied only to batches being processed, not all documents at once
- **Error Recovery**: Individual document failures don't stop the entire batch
- **Progress Tracking**: Detailed progress bars show cleaning and embedding stages
- **Checkpoint System**: Robust checkpointing allows recovery from interruptions
- **GPU Memory Management**: Automatic VRAM cleanup between batches prevents memory issues

The pipeline integrates the cleaning functions seamlessly into the batch processing workflow.

In [ ]:
# Final ingestion step - process all legislation documents through pipeline
if all_legislation_documents:
    print(f"\n🔄 Running legislation ingestion pipeline...")
    print(f"   📄 Total documents to process: {len(all_legislation_documents)}")
    print(f"   📂 Categories processed: {total_categories_processed}/{len(dir_list)}")
    print(f"   🔧 Pipeline will: split text → generate embeddings → store in ChromaDB")
    
    # Configuration for robust batch processing
    batch_size = 8  # Slightly larger batches for legislation (optimized for GPU memory)
    checkpoint_file = "./checkpoint/legislation_ingestion_checkpoint.json"
    total_batches = (len(all_legislation_documents) + batch_size - 1) // batch_size

    # Load checkpoint if exists - enables resuming from interruptions
    start_batch = 0
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, "r", encoding="utf-8", errors="ignore") as f:
            try:
                checkpoint = json.load(f)
                start_batch = checkpoint.get("last_batch", 0)
                print(f"⏩ Resuming from batch {start_batch+1}/{total_batches}")
            except Exception:
                print("⚠️ Checkpoint file corrupted, starting from batch 1")

    # Enhanced progress tracking with detailed metrics
    with tqdm(
        total=len(all_legislation_documents), 
        desc="📜 Ingesting Legislation", 
        unit="docs",
        bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]'
    ) as pbar:
        
        start_time = time.time()
        pbar.update(start_batch * batch_size)

        # Process documents in batches with real-time cleaning
        for i in range(start_batch * batch_size, len(all_legislation_documents), batch_size):
            batch = all_legislation_documents[i:i+batch_size]
            batch_num = i // batch_size + 1
            
            # 🧹 REAL-TIME DATA CLEANING: Clean each document in the batch
            cleaned_batch = []
            for doc in batch:
                try:
                    # Apply text and metadata cleaning using our sanitization functions
                    new_doc = doc.copy(update={
                        "text": clean_text(getattr(doc, "text", "")),
                        "metadata": sanitize_metadata(getattr(doc, "metadata", {}))
                    })
                    cleaned_batch.append(new_doc)
                except Exception as e:
                    print(f"⚠️ Skipping bad doc in batch {batch_num}: {e}")
                    continue
            
            # Skip empty batches (all documents failed cleaning)
            if not cleaned_batch:
                print(f"⚠️ Entire batch {batch_num} empty after cleaning, skipping")
                continue
            
            # Update progress bar with current batch information
            pbar.set_description(f"📜 Processing Batch {batch_num}/{total_batches}")
            
            try:
                # 🚀 PIPELINE PROCESSING: Run cleaned documents through embedding pipeline
                legislation_pipeline.run(documents=cleaned_batch)
            except Exception as e:
                print(f"❌ Error on batch {batch_num}: {e}")
                print("⚠️ Skipping this batch and continuing...")
                # Still save checkpoint before skipping to track progress
                with open(checkpoint_file, "w", encoding="utf-8", errors="ignore") as f:
                    json.dump({"last_batch": batch_num}, f, ensure_ascii=False)
                continue

            # 💾 CHECKPOINT SYSTEM: Save progress after successful batch processing
            with open(checkpoint_file, "w", encoding="utf-8", errors="ignore") as f:
                json.dump({"last_batch": batch_num}, f, ensure_ascii=False)

            # Update progress tracking
            pbar.update(len(batch))

            # 🧠 GPU MEMORY MANAGEMENT: Free VRAM between batches to prevent fragmentation
            import torch, gc
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            
            # Calculate and display real-time performance metrics
            elapsed_time = time.time() - start_time
            docs_per_second = pbar.n / elapsed_time if elapsed_time > 0 else 0
            pbar.set_postfix({
                'Batch': f"{batch_num}/{total_batches}",
                'Speed': f"{docs_per_second:.1f} docs/s",
                'Stage': 'Embedding',
                'Categories': f"{total_categories_processed}/{len(dir_list)}"
            })
    
    # Final success summary
    print(f"✅ Successfully ingested {pbar.n} legislation documents")
    print(f"📊 Documents now available in 'legislation' collection")
    print(f"📋 Processed {total_categories_processed} legislation categories")

else:
    print(f"\n⚠️ No legislation documents to process - skipping pipeline ingestion")


🔄 Running legislation ingestion pipeline...
   📄 Total documents to process: 40032
   📂 Categories processed: 5/5
   🔧 Pipeline will: split text → generate embeddings → store in ChromaDB


📜 Ingesting Legislation:   0%|          | 0/40032 [00:00<?, ?docs/s]C:\Users\User\AppData\Local\Temp\ipykernel_88072\2170210105.py:42: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  new_doc = doc.copy(update={
📜 Processing Batch 5004/5004: 100%|██████████| 40032/40032 [4:18:15<00:00,  2.58docs/s]   

✅ Successfully ingested 40032 legislation documents
📊 Documents now available in 'legislation' collection
📋 Processed 5 legislation categories


In [54]:
# Test Query for Legislation using proper vector store interface
if legislation_collection.count() > 0:
    print("🔍 Testing legislation query functionality with correct embedding model...")
    
    # Create a query engine using the vector store (uses your bge-m3 model)
    from llama_index.core import VectorStoreIndex
    
    # Create index from vector store
    legislation_index = VectorStoreIndex.from_vector_store(
        vector_store=legislation_vector_store,
        embed_model=embed_model  # This ensures we use YOUR embedding model
    )
    
    # Create query engine
    legislation_query_engine = legislation_index.as_query_engine(
        similarity_top_k=3
    )
    
    # Test query using the same embedding model as ingestion
    print("   🔍 Running test query with BAAI/bge-m3 model...")
    response = legislation_query_engine.query("What are the provisions for constitutional amendments?")
    
    print(f"   ✅ Query successful using {embed_model_name}!")
    print(f"   📊 Response: {response.response[:200]}...")
    print(f"   🎯 Found {len(response.source_nodes)} relevant chunks")
    
    # Show source information
    for i, node in enumerate(response.source_nodes, 1):
        doc_type = node.metadata.get('document_type', 'Unknown')
        category = node.metadata.get('category', 'Unknown')
        source = node.metadata.get('source_filename', 'Unknown')
        print(f"   📄 Source {i}: {doc_type} - {category} - {source}")
    
    print()
else:
    print("⚠️  No legislation data to test queries on")

🔍 Testing legislation query functionality with correct embedding model...
   🔍 Running test query with BAAI/bge-m3 model...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


   ✅ Query successful using BAAI/bge-m3!
   📊 Response: The provisions for constitutional amendments include the requirement for a Bill to be supported by two-thirds of the total number of members of the House of Parliament, with exceptions for certain typ...
   🎯 Found 3 relevant chunks
   📄 Source 1: legislation - Federal Constitution - federal_Constitution (Reprint 2020).pdf
   📄 Source 2: legislation - Federal Constitution - federal_Constitution (Reprint 2020).pdf
   📄 Source 3: legislation - Federal Constitution - federal_Constitution (Reprint 2020).pdf



---

## Conclusion

This notebook successfully demonstrates a complete document ingestion pipeline for legal documents. The pipeline:

✅ **Processes Multiple Document Types**: Handles both legal cases and legislation documents  
✅ **Integrates Metadata**: Combines PDF content with structured JSON metadata  
✅ **Uses Semantic Chunking**: Intelligently splits documents at semantic boundaries  
✅ **Generates Quality Embeddings**: Uses state-of-the-art BAAI/bge-m3 model  
✅ **Stores Efficiently**: Persists data in ChromaDB for fast retrieval  
✅ **Handles Errors Gracefully**: Robust error handling and logging  
✅ **Cleans Data**: Advanced text and metadata sanitization for reliable storage  
✅ **Optimizes Memory**: GPU memory management and batch processing  
✅ **Provides Recovery**: Checkpoint system enables resuming from interruptions  

### Key Features Added

**Enhanced Data Processing**:
- **Unicode Cleaning**: Removes problematic surrogate characters that cause encoding issues
- **Metadata Sanitization**: Ensures all metadata is JSON-serializable and properly sized
- **Real-time Cleaning**: Applies cleaning during batch processing for memory efficiency

**Robust Pipeline Architecture**:
- **Checkpoint System**: Allows resuming from interruptions without losing progress
- **Error Recovery**: Individual document failures don't stop the entire process
- **GPU Memory Management**: Automatic VRAM cleanup prevents memory fragmentation
- **Progress Tracking**: Detailed metrics and progress bars for monitoring

### Next Steps

After running this pipeline, the vector database will be ready for:
- **Semantic Search**: Find relevant legal documents based on natural language queries
- **Similarity Matching**: Discover related cases and legislation
- **RAG Applications**: Use as knowledge base for legal question-answering systems
- **Advanced Analytics**: Leverage clean, structured metadata for document analysis

### Database Statistics

The pipeline creates two ChromaDB collections:
- **`legal_cases`**: Contains processed court decisions and case law with cleaned metadata
- **`legislation`**: Contains acts, amendments, ordinances, and constitutional documents

Both collections now feature:
- Clean, sanitized text and metadata
- Robust error handling and recovery
- Optimized storage format
- Full compatibility with downstream applications

The enhanced pipeline ensures reliable, scalable processing of large legal document collections.